![Neptune + MosaicML Composer](https://neptune.ai/wp-content/uploads/2023/09/mosiacml.svg)

# Neptune + MosaicML

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/mosaicml-composer/notebooks/Neptune_MosaicML_Composer.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

<a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/mosaicml-composer/notebooks/Neptune_MosaicML_Composer.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a>

## Introduction

[MosaicML Composer](https://github.com/mosaicml/composer) is a PyTorch library for efficient neural network training.

This guide will show you how to:

* Create a Neptune logger for MosaicML Composer
* Automatically log your Composer training metadata to Neptune

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [1]:
%pip install -U neptune torch torchvision #mosaicml

# TODO: Update after released
%pip install git+https://github.com/AleksanderWWW/composer.git@neptune-logger

  Using cached neptune-1.7.0-py3-none-any.whl (459 kB)
  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached torchvision-0.16.0-cp310-cp310-manylinux1_x86_64.whl (6.9 MB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached PyJWT-2.8.0-py3-none-any.whl (22 kB)
  Using cached swagger_spec_validator-3.0.3-py2.py3-none-any.whl (27 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Using cached bravado-11.0.3-py2.py3-none-any.whl (38 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.6 MB/s eta 0:00:00a 0:00:01
  Using cached future-0.18.3-py3-none-any.whl
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Us

## Import libraries

In [1]:
import torch

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from composer import Trainer
from composer.models import mnist_model
from composer.loggers import NeptuneLogger
from composer.algorithms import LabelSmoothing, BlurPool, ProgressiveResizing

## Prepare dataset and dataloaders

In [2]:
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST('data', download=True, train=True, transform=transform)
eval_dataset = datasets.MNIST('data', download=True, train=False, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=128)
eval_dataloader = DataLoader(eval_dataset, batch_size=128)

## (Neptune) Create `neptune_logger`

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in the public project [common/mosaicml](https://app.neptune.ai/o/common/org/mosaicml). **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
from getpass import getpass

neptune_logger = NeptuneLogger(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
    tags=["mnist", "notebook"],  # (optional) use your own
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app.

    To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [6]:
from neptune import ANONYMOUS_API_TOKEN

neptune_logger = NeptuneLogger(
    api_token=ANONYMOUS_API_TOKEN,  # Replace with your own
    project="common/mosaicml-composer",  # Replace with your own
    capture_hardware_metrics=True,  # This is turned off by default in Notebooks
    tags=["mnist", "notebook"],  # (optional) use your own
)

https://app.neptune.ai/siddhant.sadangi/sandbox/e/SAN-1028


**To open the run in the Neptune web app, click the link that appeared in the cell output.**

## Configure Composer algorithms

### Label smoothing

In [11]:
label_smoothing = LabelSmoothing(0.1)

### Blur pooling

In [12]:
blurpool = BlurPool(
    replace_convs=True,
    replace_maxpools=True,
    blur_first=True,
)

### Progressive resizing

In [13]:
prog_resize = ProgressiveResizing(
    initial_scale=.6,
    finetune_fraction=0.34,
)

## Train model with Composer algorithms

In [14]:
trainer = Trainer(
    model=mnist_model(),
    train_dataloader=train_dataloader,
    eval_dataloader=eval_dataloader,
    max_duration="3ep",
    device="gpu" if torch.cuda.is_available() else "cpu",
    algorithms = [label_smoothing, blurpool, prog_resize],
    loggers=neptune_logger,
)

trainer.fit()

Value '2240760941' is outside the range of 32-bit integers ('-2147483648' to '2147483647') and will be logged as float


/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11080). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/opt/conda/lib/python3.10/site-packages/composer/trainer/trainer.py:982: UserWarning: No optimizer was specified. Defaulting to DecoupledSGDW(lr=0.1)
  warnings.warn(('No optimizer was specified. Defaulting to '
/opt/conda/lib/python3.10/site-packages/composer/algorithms/blurpool/blurpool.py:160: NoEffectWarning: Applying BlurPool did not change any layers. No strided Conv2d or Pool2d layers were found.
  warnings.warn(
******************************
Config:


train          Epoch   0:    0%|| 0/469 [00:00<?, ?ba/s]         

/opt/conda/lib/python3.10/site-packages/composer/loss/loss.py:134: UserWarning: Some targets have less than 1 total probability.
  warnings.warn('Some targets have less than 1 total probability.')
/opt/conda/lib/python3.10/site-packages/composer/core/data_spec.py:35: UserWarning: Cannot split tensor of length 96 into batches of size 128. As it is smaller, no splitting will be done. This may happen on the last batch of a dataset if it is a smaller size than the microbatch size.
  warnings.warn(f'Cannot split tensor of length {len(t)} into batches of size {microbatch_size}. '


eval           Epoch   0:    0%|| 0/79 [00:00<?, ?ba/s]         

...

Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: training/metrics/metrics/train/MulticlassAccuracy. Invalid point: 469.0


eval           Epoch   1:    0%|| 0/79 [00:00<?, ?ba/s]         

train          Epoch   2:    0%|| 0/469 [00:00<?, ?ba/s]         

eval           Epoch   2:    0%|| 0/79 [00:00<?, ?ba/s]         

Error occurred during asynchronous operation processing: X-coordinates (step) must be strictly increasing for series attribute: training/metrics/metrics/train/MulticlassAccuracy. Invalid point: 938.0


## Log additional metadata
`neptune_logger`'s `base_handler` property exposes the [Namespace handler](https://docs.neptune.ai/api/field_types/#namespace-handler) being used by the logger. You can use it to log any additional metadata.

**Note:** The default namespace used by `NeptuneLogger` is "training". You can update this by passing yout own "base_namespace" while initializing `NeptuneLogger`

In the below example, we log a sample of the training dataset as a series of images in the "training/images" namespace.

In [24]:
from neptune.types import File
neptune_logger.base_handler["images"].extend([File.as_image(img/255) for img in train_dataset.data[:50]])

## Log to your custom namespace
If you want to log to a different namespace than used by the base handler, you can use `neptune_logger`'s `neptune_run` property to access the underlying `Run` object and pass your own namespaces.

In the below example, we log a sample of the eval dataset as a series of images in the "eval/images" namespace.

In [21]:
neptune_logger.neptune_run["eval/images"].extend([File.as_image(img/255) for img in eval_dataset.data[:50]])

## Stop logging

Once you are done logging, stop tracking the run.

In [25]:
trainer.close()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 50 operations to synchronize with Neptune. Do not kill this process.
All 50 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/siddhant.sadangi/sandbox/e/SAN-1028/metadata


## Analyze run in the Neptune app
Follow the run link in the above cell output and explore the logged metadata.  
You can also explore this [example run](https://app.neptune.ai/o/common/org/mosaicml/runs/details?viewId=standard-view&detailsTab=dashboard&dashboardId=Overview-99f571df-0fec-4447-9ffe-5a4c668577cd&shortId=CAT-2). #TODO: Update link